<a href="https://colab.research.google.com/github/phantom0101/ICR_ensemble/blob/main/stacked_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna catboost --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

# from loss.log_loss import balanced_log_loss, calc_log_loss_weight
# from opt.optweights import OptunaWeights
import optuna
from functools import partial
# from pprintpp import pprint
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import catboost as cat
import lightgbm as lgb



n_estimators=100
# device="cpu"
# random_state=state
n_splits= 5
# random_state = 71
# n_estimators = 99999
verbose = False
early_stopping_rounds= 1000

In [3]:
def balanced_log_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    balanced_log_loss_score = (-w0/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(1-y_pred))) - w1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred)))) / (w0+w1)
    return balanced_log_loss_score

def calc_log_loss_weight(y_true):
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1

In [4]:
class OptunaWeights:
    def __init__(self, random_state, n_trials=3):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 1e-14, 1) for n in range(len(y_preds))]

        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds).T
                                   , axis=1
                                   , weights=weights)
        # np.average(np.array(y_preds).T, axis=1, weights=weights)

        # Calculate the score for the weighted prediction
        # score = log_loss(y_true, weighted_pred)
        score = balanced_log_loss(y_true, weighted_pred)
        return score

    def fit(self, y_true, y_preds):
        # print(np.array(y_preds).T)
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        pruner = optuna.pruners.HyperbandPruner()
        self.study = optuna.create_study(
            sampler=sampler, pruner=pruner, 
            study_name="OptunaWeights", direction='minimize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]

    def predict(self, y_preds):
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds):
        self.fit(y_true, y_preds)
        return self.predict(y_preds)
    
    def weights(self):
        return self.weights
    
    


In [5]:
def get_params(state):
    random_state = state
    n_estimators = 100
    # device = 'cpu'
    # xgb_params = {
    #     'n_estimators': n_estimators,
    #     'learning_rate': 0.413327571405248,
    #     'booster': 'gbtree',
    #     'lambda': 0.0000263894617720096,
    #     'alpha': 0.000463768723479341,
    #     'subsample': 0.237467672874133,
    #     'colsample_bytree': 0.618829300507829,
    #     'max_depth': 5,
    #     'min_child_weight': 9,
    #     'eta': 2.09477807126539E-06,
    #     'gamma': 0.000847289463422307,
    #     'grow_policy': 'depthwise',
    #     'n_jobs': -1,
    #     'objective': 'binary:logistic',
    #     'eval_metric': 'logloss',
    #     'verbosity': 0,
    #     'random_state': random_state,
        
    #     'early_stopping_rounds': 1000,
    #     'scale_pos_weight': 4.71,
    # }
    """"optuna"""
    xgb_params={'n_estimators': 85, 
        'learning_rate': 0.5532875740096689,
        'booster': 'dart',
        'lambda': 2.210583397046728e-05,
        'alpha': 0.00011961099653447517,
        'subsample': 0.16149076133637538,
        'scale_pos_weight': 2.3999263887936273,
        'subsample': 0.237467672874133,
        'colsample_bytree': 0.618829300507829,
        'max_depth': 5,
        'min_child_weight': 9,
        'eta': 2.09477807126539E-06,
        'gamma': 0.000847289463422307,
        'grow_policy': 'depthwise',
        'n_jobs': -1,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0,
        'random_state': random_state,
        'early_stopping_rounds': 1000,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        }
    # if device == 'gpu':
    #     xgb_params['tree_method'] = 'gpu_hist'
    #     xgb_params['predictor'] = 'gpu_predictor'
            
    lgb_params = {
        'n_estimators': n_estimators,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'num_leaves': 5,
        'colsample_bytree': 0.50,
        'subsample': 0.80,
        'reg_alpha': 2, 
        'reg_lambda': 4,
        'n_jobs': -1,
        'is_unbalance':True,
        # 'device': 'gpu',
        'random_state': random_state,
        'verbose': -1,
        'early_stopping_round': early_stopping_rounds,
        
        'class_weight': 'balanced'
    }
    """optuna"""
    # lgb_params = {
    #     'n_estimators': n_estimators,
    #     'objective': 'binary',
    #     'boosting_type': 'gbdt',
    #     'learning_rate': 0.,
    #     'num_leaves': 5,
    #     'colsample_bytree': 0.50,
    #     'subsample': 0.80,
    #     'reg_alpha': 2.7, 
    #     'reg_lambda': 4.65,
    #     'n_jobs': -1,
    #     'is_unbalance':True,
    #     # 'device': device,
    #     'random_state': random_state,
    #     'verbose': -1,
    #     'early_stopping_round': early_stopping_rounds,
        
    #     'class_weight': 'balanced'
    # }
    lgb1_params = {
        'n_estimators': n_estimators,
        'learning_rate': 0.190197487721534,
        'reg_alpha': 0.00749112221417973,
        'reg_lambda': 0.000548118227209224,
        'num_leaves': 17,
        'colsample_bytree': 0.547257860506146,
        'subsample': 0.592628085686409,
        'subsample_freq': 2,
        'min_child_samples': 64,
        'objective': 'binary',
        #'metric': 'binary_error',
        'boosting_type': 'gbdt',
        'is_unbalance':True,
        # 'device': self.device,
        'random_state': random_state
    } 
    # lgb2_params = {
    #     'n_estimators': self.n_estimators,
    #     'learning_rate': 0.181326407627473,
    #     'reg_alpha': 0.000030864084239014,
    #     'reg_lambda': 0.0000395714763869486,
    #     'num_leaves': 122,
    #     'colsample_bytree': 0.75076596295323,
    #     'subsample': 0.6303245788342,
    #     'subsample_freq': 3,
    #     'min_child_samples': 72,
    #     'objective': 'binary',
    #     #'metric': 'binary_error',
    #     'boosting_type': 'gbdt',
    #     'is_unbalance':True,
    #     'device': self.device,
    #     'random_state': self.random_state
    # } 
    cat_params = {
        'iterations': n_estimators,
        'colsample_bylevel': 0.0513276895988184,
        'depth': 2,
        'learning_rate': 0.0256579773375401,
        'l2_leaf_reg': 8.22319805476255,
        'random_strength': 0.11327724457066,
        'od_type': "Iter", 
        'od_wait': 72,
        'bootstrap_type': "Bayesian",
        'grow_policy': 'SymmetricTree',
        'bagging_temperature': 9.58737431845122,
        #'eval_metric': 'Logloss',
        #'loss_function': 'Logloss',
        'auto_class_weights': 'Balanced',
        # 'task_type': 'GPU',
        'random_state': random_state,
        
        # 'early_stopping_rounds': 1000,
    }
    # hist_params = {
    #     'l2_regularization': 0.01,
    #     'early_stopping': True,
    #     'learning_rate': 0.01,
    #     'max_iter': self.n_estimators,
    #     'max_depth': 4,
    #     'max_bins': 255,
    #     'min_samples_leaf': 10,
    #     'max_leaf_nodes':10,
    #     'class_weight':'balanced',
    #     'random_state': self.random_state
    # }
    return xgb_params, cat_params, lgb_params, lgb1_params


In [94]:
def act_main(state, weights, test_pred, oof_ens_pred):
    random_state = state
    
    xgb_params, cat_params, lgb_params, _ = get_params(state)
    train = pd.read_csv("/content/processed_train.csv")
    test = pd.read_csv("/content/processed_test.csv")

    X = train.drop('Class', axis=1)
    y = train['Class']
    
    trainx, testx, trainy, testy = train_test_split(X, y , test_size=0.1, random_state=random_state)

    
    clf1 = xgb.XGBClassifier(**xgb_params)
    clf2 = cat.CatBoostClassifier(**cat_params)
    clf3 = lgb.LGBMClassifier(**lgb_params)
    
    skf = StratifiedKFold(n_splits=n_splits)
    
    oof_predss = np.zeros((trainx.shape[0], 5))
    xgb_oof_pred = []
    cat_oof_pred = []
    lgb_oof_pred = []
    fold_target = []
    # oof_ens_pred = []
    oof_loss = []
    ensemble_score = []
    test_pred = []

    for fold, (train_idx, valid_idx) in tqdm(enumerate(skf.split(trainx, trainy))):
        xtrain = trainx.iloc[train_idx]
        ytrain = trainy.iloc[train_idx]
        
        xvalid = trainx.iloc[valid_idx]
        yvalid = trainy.iloc[valid_idx]
        
        train_w0, train_w1 = calc_log_loss_weight(ytrain)
        valid_w0, valid_w1 = calc_log_loss_weight(yvalid)
        
        
        clf1.fit(
                xtrain, ytrain, sample_weight=ytrain.map({0: train_w0, 1: train_w1}), 
                eval_set=[(xvalid, yvalid)], sample_weight_eval_set=[yvalid.map({0: valid_w0, 1: valid_w1})],
                # early_stopping_rounds=early_stopping_rounds, 
                verbose=verbose)
        
        clf2.fit(cat.Pool(xtrain, ytrain, weight=ytrain.map({0: train_w0, 1: train_w1})), 
                eval_set=cat.Pool(xvalid, yvalid, weight=yvalid.map({0: valid_w0, 1: valid_w1})), 
                early_stopping_rounds=early_stopping_rounds, verbose=verbose)
        
        clf3.fit(xtrain, ytrain, sample_weight=ytrain.map({0: train_w0, 1: train_w1}), 
                eval_set=[(xvalid, yvalid)], eval_sample_weight=[yvalid.map({0: valid_w0, 1: valid_w1})],
                callbacks=[lgb.log_evaluation(0)])
        
        xgb_pred = clf1.predict_proba(xvalid)[:, 1].reshape(-1)
        cat_pred = clf2.predict_proba(xvalid)[:, 1].reshape(-1)
        lgb_pred = clf3.predict_proba(xvalid)[:, 1].reshape(-1)
        ens_pred = (cat_pred + xgb_pred + lgb_pred)/3
        
        # cat_loss = balanced_log_loss(cat_pred, yvalid)
        # xgb_loss = balanced_log_loss(xgb_pred, yvalid)
        # lgb_loss = balanced_log_loss(lgb_pred, yvalid)
        ens_loss = balanced_log_loss(yvalid, ens_pred)
        oof_loss.append(ens_loss)
        oof_ens_pred.append(np.array(xgb_pred))
        oof_ens_pred.append(np.array(cat_pred))
        oof_ens_pred.append(np.array(lgb_pred))
        
        
        # oof_ens_pred.append(ens_pred)
        
        # print(np.array(oof_ens_pred).T)
        xgb_test_pred = clf1.predict_proba(test)[:, 1].reshape(-1)
        cat_test_pred = clf2.predict_proba(test)[:, 1].reshape(-1)
        lgb_test_pred = clf3.predict_proba(test)[:, 1].reshape(-1)
        # test_test_pred = model.predict_proba(test)[:, 1].reshape(-1)
        test_pred.append(xgb_test_pred)
        test_pred.append(cat_test_pred)
        test_pred.append(lgb_test_pred)
        
        """OPTUNA"""
        # data = pd.read_csv("ICR/ICR_data/pred_for_lgb1.csv")
        # oof_preds = data.drop('Class', axis=1)
        # y_val = data.Class

        optweights = OptunaWeights(random_state=random_state)
        y_val_pred = optweights.fit_predict(yvalid.values, oof_ens_pred)
        
        
        score = balanced_log_loss(yvalid, y_val_pred)
        # score_ = roc_auc_score(y_val, y_val_pred)
        print(f'--> Ensemble [FOLD-{fold}] BalancedLogLoss score {score:.5f}')
        ensemble_score.append(score)
        # ensemble_score_ = [score_]
    weights.append(optweights.weights)
    # print(optweights.weights)
    test_each_pred.append(test_pred)
    # oof_each_pred.append(oof_ens_pred)
        
    print("*"*45)
    # print(f"mean_oof_loss: {np.mean(oof_loss)}")
    print(f"mean_cv_loss: {np.mean(ensemble_score)}")


    """"test score"""
    # ens_test_pred = []
    xgb_test_pred = clf1.predict_proba(testx)[:, 1].reshape(-1)
    cat_test_pred = clf2.predict_proba(testx)[:, 1].reshape(-1)
    lgb_test_pred = clf3.predict_proba(testx)[:, 1].reshape(-1)
    ens_test_pred = (cat_test_pred + xgb_test_pred + lgb_test_pred)/3
    # ens_test_pred.append(xgb_test_pred)
    # ens_test_pred.append(cat_test_pred)
    # ens_test_pred.append(lgb_test_pred)
    # weighted_ens_test_pred = np.average(np.array(ens_test_pred).T, axis=1, weights=np.mean(np.array(weights).T))
    print(f"ens_test_pred_loss: {balanced_log_loss(testy, ens_test_pred)}")
    # pred = cat_oof_pred + xgb_oof_pred + fold_target
        
    # pd.DataFrame(pred).T.to_csv("ICR/ICR_data/oof_pred.csv", index=False)


    #     test = pd.red_csv("/kaggle/working/processed_test.csv")
    #     xgb = clf1.predict_proba(testx)[:, 1].reshape(-1)
    #     cat = clf2.predict_proba(testx)[:, 1].reshape(-1)
    #     lgb = clf3.predict_proba(testx)[:, 1].reshape(-1)
    #     ens = (cat_test_pred + xgb_test_pred + lgb_test_pred)/3
    return ens_test_pred, weights, test_pred, oof_ens_pred


In [103]:
score = []
weights = []
weights_each_state = []
oof_ens_each_pred = []
test_each_pred = []
# test_each_preds = np.zeros((test.shape[0], 3))
for state in range(1000, 3000, 1000):
    test_pred = []
    oof_each_pred = []
    ens, wei, test_pred, oof_each_pred = act_main(state, weights, test_pred, oof_each_pred)
    score.append(ens)
    weights_each_state.append(wei)
    test_each_pred.append(test_pred)
    oof_ens_each_pred.append(oof_each_pred)
    print(oof_ens_each_pred)  

# print(np.mean(score))

1it [00:02,  2.45s/it]

--> Ensemble [FOLD-0] BalancedLogLoss score 0.19045


2it [00:05,  2.89s/it]

--> Ensemble [FOLD-1] BalancedLogLoss score 0.48912


3it [00:08,  2.69s/it]

...

--> Ensemble [FOLD-2] BalancedLogLoss score 0.60332


4it [00:10,  2.61s/it]

--> Ensemble [FOLD-3] BalancedLogLoss score 0.60201


5it [00:13,  2.62s/it]

--> Ensemble [FOLD-4] BalancedLogLoss score 0.62447
*********************************************
mean_cv_loss: 0.5365747964151576
ens_test_pred_loss: 0.24671622258102466
[[array([1.27031759e-03, 2.08512112e-01, 9.82344866e-01, 5.32157183e-01,
       2.86441832e-03, 6.57237973e-03, 6.63402230e-02, 1.10745005e-01,
       2.63287462e-02, 2.98059545e-02, 9.28440690e-03, 1.49093539e-01,
       3.76455719e-04, 6.44341931e-02, 2.97923442e-02, 5.63249946e-01,
       2.43594244e-01, 4.57507838e-03, 2.44068932e-02, 1.86480388e-01,
       1.80672668e-02, 6.27334416e-03, 6.00245744e-02, 2.90225238e-01,
       2.04146490e-01, 1.10187558e-02, 2.27877907e-02, 6.44423347e-03,
       9.92553413e-01, 9.89912450e-01, 9.25950825e-01, 3.35119031e-02,
       4.47426021e-01, 3.95989453e-04, 2.83523276e-02, 1.10405022e-02,
       1.01807748e-03, 2.00690195e-01, 5.37948236e-02, 7.25010872e-01,
       6.69011891e-01, 9.95505452e-01, 3.45881172e-02, 5.97764969e-01,
       6.93925261e-01, 3.09922546e-01, 4.59401

In [106]:
# np.mean(np.mean(weights, axis=0).reshape(5, 3), axis=0)
temp = np.array([np.array(i) for i in oof_ens_each_pred])
# len(oof_ens_each_pred[0])



(2, 15, 111)

In [96]:
# temp = np.array(oof_ens_each_pred)
# for i in len(oof_ens_each_pred):
# temp2 = np.array([np.mean((oof_ens_each_pred[i]), axis=0) for i in range(len(oof_ens_each_pred))])
# np.mean(temp, axis=0)
# print([np.mean(i, axis=0) for i in temp2])
temp.shape

(2,)

In [93]:
np.mean(temp, axis=0)


ValueError: ignored

In [58]:
normalize = [((weight - np.min(weight)) / (np.max(weight) - np.min(weight))).tolist() for weight in weights]
np.mean(np.mean(normalize, axis=0).reshape(5, 3), axis=0)

array([0.56337537, 0.52057715, 0.53690459])

In [88]:
# (np.mean(np.array(np.mean(temp[0])), axis=0), np.mean(np.array(np.mean(temp[1])), axis=0))
np.mean(temp[0], axis=0)

array([[0.50137921, 0.48788764, 0.53263701, ..., 0.44724333, 0.34221576,
        0.23258901],
       [0.52415984, 0.51140124, 0.55528035, ..., 0.4179684 , 0.36623485,
        0.22776225],
       [0.50500028, 0.53168179, 0.55941067, ..., 0.42047543, 0.36794897,
        0.225529  ],
       ...,
       [0.55556698, 0.51082776, 0.58923192, ..., 0.3881229 , 0.34345201,
        0.20394515],
       [0.5285483 , 0.50013764, 0.5604825 , ..., 0.41260618, 0.36349382,
        0.22660499],
       [0.5564132 , 0.51264059, 0.58682167, ..., 0.3940528 , 0.34314667,
        0.22914245]])

In [87]:
train = pd.read_csv("/content/processed_train.csv")
test = pd.read_csv("/content/processed_test.csv")
stack_test_predss = np.zeros((test.shape[0]))
stack_scores = []
stack_models = []
y = train['Class']


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=state)
# splitter = Splitter(kfold=kfold, n_splits=n_splits, greeks=greeks.iloc[:,1:-1])
for state in range(1000, 3000, 1000):
    _, _, _, lgb1_param = get_params(state)
    m = state//1000 - 1 
    # print(np.mean(np.array(np.mean(temp[m])), axis=0))
    # print(y)
    for i, (X_train_, X_val, y_train_, y_val) in enumerate(skf.split(np.array(np.mean(temp[m], axis=0)), np.array(y))):
        
        model = lgb.LightGBMClassifier(**lgb1_param)
        
        train_w0, train_w1 = calc_log_loss_weight(y_train_)
        valid_w0, valid_w1 = calc_log_loss_weight(y_val)
        
        model.fit(
            X_train_, y_train_, sample_weight=y_train_.map({0: train_w0, 1: train_w1}), 
            eval_set=[(X_val, y_val)], eval_sample_weight=[y_val.map({0: valid_w0, 1: valid_w1})],
            early_stopping_rounds=early_stopping_rounds, verbose=verbose)
    
        test_pred = model.predict_proba(test_each_pred)[:, 1].reshape(-1)
        y_val_pred = model.predict_proba(X_val)[:, 1].reshape(-1)

        score = balanced_log_loss(y_val, y_val_pred)
        stack_scores.append(score)
        # stack_models.append(deepcopy(model))
        
        stack_test_predss += test_pred / (n_splits * len(2))

ValueError: ignored